In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\Shreyansh Singh\Downloads\Weather_Data.csv")
df.head(10)


In [13]:
df['Date/Time'] =pd.to_datetime(df['Date/Time']
                                )
df.set_index('Date/Time' , inplace = True)

In [14]:
df.head(10)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
Date/Time,,,,,,,
2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
2012-01-01 03:00:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
2012-01-01 04:00:00,-1.5,-3.3,88,7,4.8,101.23,Fog
2012-01-01 05:00:00,-1.4,-3.3,87,9,6.4,101.27,Fog
2012-01-01 06:00:00,-1.5,-3.1,89,7,6.4,101.29,Fog
2012-01-01 07:00:00,-1.4,-3.6,85,7,8.0,101.26,Fog
2012-01-01 08:00:00,-1.4,-3.6,85,9,8.0,101.23,Fog


In [15]:
df.columns

Index(['Temp_C', 'Dew Point Temp_C', 'Rel Hum_%', 'Wind Speed_km/h',
       'Visibility_km', 'Press_kPa', 'Weather'],
      dtype='object')

In [28]:
features = ['Temp_C' , 'Dew Point Temp_C' , 'Rel Hum_%' , 'Wind Speed_km/h' , 'Visibility_km' , 'Press_kPa']
from sklearn.preprocessing import MinMaxScaler
df[features] = df[features].fillna(method = 'ffill')
scaler = MinMaxScaler()
scaled_data= scaler.fit_transform(df[features])
import joblib
joblib.dump(scaler , 'scaler_lstm.pkl')


['scaler_lstm.pkl']

In [29]:
df.head()

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
Date/Time,,,,,,,
2012-01-01 00:00:00,0.381883,0.465028,0.829268,0.048193,0.162162,0.606852,Fog
2012-01-01 01:00:00,0.381883,0.468809,0.841463,0.048193,0.162162,0.606852,Fog
2012-01-01 02:00:00,0.381883,0.474480,0.865854,0.084337,0.079002,0.610114,"Freezing Drizzle,Fog"
2012-01-01 03:00:00,0.387211,0.478261,0.853659,0.072289,0.079002,0.611746,"Freezing Drizzle,Fog"
2012-01-01 04:00:00,0.387211,0.476371,0.853659,0.084337,0.095634,0.605220,Fog


In [30]:

def create_sequences(data, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        labels.append(data[i + sequence_length, 0])  # Assuming the temperature is the first column
    return np.array(sequences), np.array(labels)

# Define the sequence length
sequence_length = 3
x, y = create_sequences(scaled_data, sequence_length)

# Splitting the data into training and testing sets
split = int(0.8 * len(x))
x_train, x_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

# Print to verify
print(x_train)

[[[0.38188277 0.46502836 0.82926829 0.04819277 0.16216216 0.60685155]
  [0.38188277 0.46880907 0.84146341 0.04819277 0.16216216 0.60685155]
  [0.38188277 0.47448015 0.86585366 0.08433735 0.07900208 0.61011419]]

 [[0.38188277 0.46880907 0.84146341 0.04819277 0.16216216 0.60685155]
  [0.38188277 0.47448015 0.86585366 0.08433735 0.07900208 0.61011419]
  [0.38721137 0.47826087 0.85365854 0.07228916 0.07900208 0.61174551]]

 [[0.38188277 0.47448015 0.86585366 0.08433735 0.07900208 0.61011419]
  [0.38721137 0.47826087 0.85365854 0.07228916 0.07900208 0.61174551]
  [0.38721137 0.47637051 0.85365854 0.08433735 0.0956341  0.60522023]]

 ...

 [[0.64653641 0.7637051  0.90243902 0.20481928 0.26403326 0.50897227]
  [0.64476021 0.76748582 0.92682927 0.18072289 0.16216216 0.49429038]
  [0.65008881 0.76748582 0.90243902 0.15662651 0.1975052  0.47797716]]

 [[0.64476021 0.76748582 0.92682927 0.18072289 0.16216216 0.49429038]
  [0.65008881 0.76748582 0.90243902 0.15662651 0.1975052  0.47797716]
  [0.6

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM

model = Sequential()
model.add(LSTM(50 , return_sequences = True , input_shape = ( sequence_length, x_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer = 'adam' , loss = 'mean_squared_error')

history = model.fit(x_train , y_train , epochs = 50 , validation_split = 0.2 , batch_size = 32)

summary = model.summary()
print(summary)

Epoch 1/50
176/176 [==============================] - 6s 11ms/step - loss: 0.0187 - val_loss: 0.0010
Epoch 2/50
176/176 [==============================] - 1s 6ms/step - loss: 0.0010 - val_loss: 9.9260e-04
Epoch 3/50
176/176 [==============================] - 1s 6ms/step - loss: 9.1870e-04 - val_loss: 8.8419e-04
Epoch 4/50
176/176 [==============================] - 1s 7ms/step - loss: 8.6402e-04 - val_loss: 7.8074e-04
Epoch 5/50
176/176 [==============================] - 1s 7ms/step - loss: 8.1152e-04 - val_loss: 7.3679e-04
Epoch 6/50
176/176 [==============================] - 1s 7ms/step - loss: 7.8405e-04 - val_loss: 7.1657e-04
Epoch 7/50
176/176 [==============================] - 1s 7ms/step - loss: 7.3102e-04 - val_loss: 6.2784e-04
Epoch 8/50
176/176 [==============================] - 1s 7ms/step - loss: 6.8717e-04 - val_loss: 5.7360e-04
Epoch 9/50
176/176 [==============================] - 1s 6ms/step - loss: 6.4650e-04 - val_loss: 5.6087e-04
Epoch 10/50
176/176 [==================

In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 3, 50)             11400     
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 31651 (123.64 KB)
Trainable params: 31651 (123.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
loss = model.evaluate(x_test , y_test)

predictions  = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

55/55 [==============================] - 1s 3ms/step


ValueError: non-broadcastable output operand with shape (1757,1) doesn't match the broadcast shape (1757,6)